In [7]:
import io
import json
import base64
from functools import partial
from pathlib import Path
import numpy as np
from skimage import draw
from skimage.io import imshow

import imageio

import pandas as pd
from bqplot import (
    Figure,
    Scatter,
    LinearScale,
    Axis,
    Toolbar,
    Label,
    ColorAxis,
    ColorScale,
    OrdinalColorScale
)
import ipywidgets as widgets
from IPython.display import display
from omero_utils.widgets import OMEConnect
from omero_utils.roi_utils import get_roi_thumb, html_thumb


In [2]:
m = OMEConnect(host="139.124.81.38")
m

OMEConnect(children=(Text(value='', description='OME loggin'), Password(description='OME password'), Button(de…

In [3]:
conn = m.conn

image = conn.getObject("Image", 18651)

roi_service = conn.getRoiService()
rois = roi_service.findByImage(image.getId(), None).rois
roi = rois[0]




In [4]:
data_dir = Path("/home/guillaume/AmuBox/centuribox-guillaume/DataScatterplots/from zong/")
im = Path(image.getName())
data_file = data_dir / (im.as_posix()[:-len(im.suffix)]+".csv")
data = pd.read_csv(data_file)

In [13]:


class ScatterViz(widgets.VBox):
    
    def __init__(self, conn, image, measures, port=4080):
        
        self.image = image
        self.measures = measures
        self.sheet = widgets.Output()
        self.conn = conn
        if not conn.isConnected():
            conn.connect()
        roi_service = conn.getRoiService()
        print(conn.isConnected())
        self.rois = roi_service.findByImage(image.getId(), None).rois
        self.thumbs = {}
        self.goto = widgets.HTML('')
        x_sc = LinearScale()
        y_sc = LinearScale()
        c_sc = ColorScale(scheme="viridis")
    
        self.scat = Scatter(
            x=self.measures["Area"],
            y=self.measures["Circ."],
            color=self.measures['AR'],
            scales={
                "x": x_sc,
                "y": y_sc,
                "color": c_sc,
            },
            names=self.measures["n_ROI"],
            display_names=False,
            fill=True,
            default_opacities=[0.8,],
        )
        ax_x = Axis(
            scale=x_sc,
            label="Area",
            tick_format='0.2f'
        )
        ax_y = Axis(
            scale=y_sc,
            label='Circularity',
            orientation="vertical"
        )
    
        self.scat.on_element_click(self.goto_db)
        self.scat.on_element_click(self.show_data)
        self.scat.on_hover(self.show_thumb)
        self.scat.tooltip = widgets.HTML("")

        self.fig = Figure(
            marks=[self.scat,],
            axes=[ax_x, ax_y],
        )
        self.base_url = f"""http://{self.conn.host}:{port}/webclient/img_detail/{self.image.id}/"""
        super().__init__([self.fig, widgets.HBox([self.goto, self.sheet])])
        

    def get_thumb(self, idx):
        th = self.thumbs.get(idx)
        if th is None:
            roi = self.rois[idx]
            th = html_thumb(get_roi_thumb(self.conn, self.image, roi, draw_roi=True))
            self.thumbs[idx] = th
        return th
        
    def show_thumb(self, cbk, target):
        name = target['data']["name"]
        self.scat.tooltip = widgets.HTML(self.get_thumb(name))
        sv.scat.tooltip.style = {"opacity": 1.0}
        
    def goto_db(self, cbk, target):
        name = target['data']["name"]
        html_ = self.get_thumb(name)
        try:
            pixel_size = self.image.getPrimaryPixels().getPhysicalSizeX().getValue()
        except AttributeError:
            pixel_size = 1
        current = self.measures.query(f"n_ROI=={name}")
        xc, yc = int(current.X / pixel_size), int(current.Y / pixel_size)
        coords = f"?x={xc}&y={yc}&zm=400"
        colors = "&c=1|0:255$FF0000,2|0:255$00FF00,3|0:255$0000FF,-4|0:255$FF0000&m=c"
        url = self.base_url + coords + colors
        self.goto.value = (f'<p><hr></p><a href={url} target="_blank">{html_}</a>')

    def show_data(self, cbk, target):
        self.sheet.clear_output()
        name = target['data']["name"]
        with self.sheet:
            display(self.measures[self.measures["n_ROI"]==name].T)
                        


In [14]:
m.conn.connect()
image = m.conn.getObject("Image", 18651)

sv = ScatterViz(m.conn, image, data)
sv

True


ScatterViz(children=(Figure(axes=[Axis(label='Area', scale=LinearScale(), tick_format='0.2f'), Axis(label='Cir…

In [77]:
sv.scat.tooltip


HTML(value='')

In [62]:
roi = sv.rois[0]


In [66]:
roi.getDescription()

In [70]:

roi.id

object #0 (::omero::RLong)
{
    _val = 9201
}